In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-__9i11a7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-__9i11a7
  Resolved https://github.com/huggingface/transformers to commit ffb35fe14283d61cd5434a32d04d07993e66477a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.0/515.0 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.1 MB/s eta 0:00:00:00:01
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11371003 sha256=25c061d5b00794a95d89073158cbddf628dfcab0e2ebe5b63170c6686dfa249a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hgv7lc3j/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Fo

In [2]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-8B-Instruct", dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-8B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-8B-Instruct")


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [3]:
def inference(image, prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    
    # Preparation for inference
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to(model.device)
    
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=1280)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    for text in output_text:
        print(text)

In [4]:
from PIL import Image

def resize_image(image_path, height, width):
    new_size = (height, width)
    image = Image.open(image_path)
    resized_image = image.resize(new_size, Image.LANCZOS)
    return resized_image 

### resized 1024,1024, markdown

In [8]:
image = '/kaggle/input/psc-forms/20251111_160053.jpg'
resized_image = resize_image(image, 1024, 1024)
prompt = 'Convert to markdown'
inference(resized_image, prompt)

```markdown
REPUBLIC OF MAURITIUS

# PUBLIC SERVICE COMMISSION
## Application Form

P.S.C. Form 7  
FOR OFFICIAL USE

---

**1. Post Applied for:** Administrative Officer

- Ministry/Department: Ministry of Finance
- Date of Advertisement/Vacancy Circular: 05/05/2025
- Previous Posts Applied for:
  1. Management Analyst
  2. Executive Assistant

---

**2. National Identity No.**  
`0851123456789`

- Title: Mr ☐ Mrs ☐ Miss ☑ Ms ☐
- Marital Status: Married ☐ Single ☑ Other: ___________
- Surname: **Scarlett** (in block letters)
- Other Names: **Silverstone** (in block letters)
- Maiden Name (if applicable): **Not Applicable**

---

**3. Residential Address:**  
NO. 15, FLOWER LANE  
COLONBO 07

- Phone No.: Office: 01234567, Home: 07712345, Mobile: 57712346
- Email address: **Scarlett@gmail.com**
- Date of Birth: 15/08/1985
- Age: 30
- Place of Birth: Colombo
- Nationality: American
- Certificate No. (If Naturalised): NLA
- Date: 11/11/2025

---

**4. SECONDARY ORDINARY LEVEL**

State wh

### resized 1024,1024, json

In [9]:
image = '/kaggle/input/psc-forms/20251111_160053.jpg'
resized_image = resize_image(image, 1024, 1024)
prompt = """
Extract all visible text and structured content from this image or PDF.
Do not omit any textual element, even if faint or partially visible.
If handwriting~ or unclear text appears, write [unreadable] or your best guess in brackets.
{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "PSCApplication",
  "type": "object",
  "properties": {
    "post_applied_for": { "type": "string" },
    "ministry_department": { "type": "string" },
    "date_of_advertisement": { "type": "string", "format": "date" },
    "national_identity_no": { "type": "string" },
    "surname": { "type": "string" },
    "other_names": { "type": "string" },
    "residential_address": { "type": "string" },
    "date_of_birth": { "type": "string", "format": "date" },
    "age": { "type": "integer" },
    "place_of_birth": { "type": "string" },
    "nationality": { "type": "string" },
    "phone_mobile": { "type": "string" },
    "email": { "type": "string", "format": "email" },
 
    "degree_qualifications": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "level": { "type": "string" },
          "institution": { "type": "string" },
          "country": { "type": "string" },
          "qualification_name": { "type": "string" },
          "class_division_level": { "type": "string" },
          "date_of_result": { "type": "string", "format": "date" },
          "course_type": { "type": "string" },
          "duration_from": { "type": "string", "format": "date" },
          "duration_to": { "type": "string", "format": "date" }
        },
        "required": [
          "level", "institution", "country", "qualification_name",
          "class_division_level", "date_of_result", "course_type",
          "duration_from", "duration_to"
        ]
      }
    },
 
    "diploma_qualifications": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "level": { "type": "string" },
          "institution": { "type": "string" },
          "country": { "type": "string" },
          "qualification_name": { "type": "string" },
          "class_division_level": { "type": "string" },
          "date_of_result": { "type": "string", "format": "date" },
          "course_type": { "type": "string" },
          "duration_from": { "type": "string", "format": "date" },
          "duration_to": { "type": "string", "format": "date" }
        },
        "required": [
          "level", "institution", "country", "qualification_name",
          "class_division_level", "date_of_result", "course_type",
          "duration_from", "duration_to"
        ]
      }
    },
 
    "ordinary_level_exams": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "exam_type": { "type": "string" },
          "month_year": { "type": "string" },
          "index_no": { "type": "string" },
          "exam_centre_no": { "type": "string" },
          "subjects": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "subject": { "type": "string" },
                "grade": { "type": "string" }
              },
              "required": ["subject", "grade"]
            }
          },
          "result": { "type": "string" },
          "aggregate": { "type": "string" }
        },
        "required": [
          "exam_type", "month_year", "index_no",
          "exam_centre_no", "subjects", "result", "aggregate"
        ]
      }
    },
 
    "advanced_level_exams": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "exam_type": { "type": "string" },
          "month_year": { "type": "string" },
          "index_no": { "type": "string" },
          "exam_centre_no": { "type": "string" },
          "subjects": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "subject": { "type": "string" },
                "grade": { "type": "string" },
                "level": { "type": "string" }
              },
              "required": ["subject", "grade", "level"]
            }
          },
          "result": { "type": "string" },
          "aggregate": { "type": "string" }
        },
        "required": [
          "exam_type", "month_year", "index_no",
          "exam_centre_no", "subjects", "result", "aggregate"
        ]
      }
    },
 
    "current_government_employment": {
      "type": "object",
      "properties": {
        "post_held": { "type": "string" },
        "temporary_substantive": { "type": "string" },
        "ministry_department": { "type": "string" },
        "date_of_appointment": { "type": "string", "format": "date" },
        "date_of_confirmation": { "type": "string", "format": "date" },
        "present_salary": { "type": "number" }
      },
      "required": [
        "post_held", "temporary_substantive", "ministry_department",
        "date_of_appointment", "date_of_confirmation", "present_salary"
      ]
    },
 
    "other_employment": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "post_held": { "type": "string" },
          "ministry_department": { "type": "string" },
          "date_of_appointment": { "type": "string", "format": "date" },
          "present_salary": { "type": "number" }
        },
        "required": ["post_held", "ministry_department", "date_of_appointment", "present_salary"]
      }
    },
 
    "investigation_enquiry": { "type": "boolean" },
    "investigation_details": { "type": ["string", "null"] },
    "court_conviction": { "type": "boolean" },
    "conviction_details": { "type": ["string", "null"] },
    "resigned_retired_dismissed": { "type": "boolean" },
    "resignation_details": { "type": ["string", "null"] },
 
    "other_qualifications": {
      "type": "array",
      "items": { "type": "string" }
    }
  },
  "required": [
    "post_applied_for",
    "ministry_department",
    "date_of_advertisement",
    "national_identity_no",
    "surname",
    "other_names",
    "residential_address",
    "date_of_birth",
    "age",
    "place_of_birth",
    "nationality",
    "phone_mobile",
    "email"
  ]
}"""

inference(resized_image, prompt)

{
  "post_applied_for": "Administrative Officer",
  "ministry_department": "Ministry of Finance",
  "date_of_advertisement": "2025-05-15",
  "national_identity_no": "Q85112345678C",
  "surname": "Scarlett",
  "other_names": "Silverstone",
  "residential_address": "NO.15, FLOWER LANE, COLONBO 07",
  "date_of_birth": "1995-05-15",
  "age": 30,
  "place_of_birth": "Colombo",
  "nationality": "American",
  "phone_mobile": "0771234567",
  "email": "Scarlett@gmail.com",
  "degree_qualifications": [],
  "diploma_qualifications": [],
  "ordinary_level_exams": [
    {
      "exam_type": "Cambridge S.C.",
      "month_year": "2011-0",
      "index_no": "110123",
      "exam_centre_no": "LK0112",
      "subjects": [
        {
          "subject": "Mathematics",
          "grade": "A"
        },
        {
          "subject": "Science",
          "grade": "A"
        },
        {
          "subject": "English Language",
          "grade": "A"
        },
        {
          "subject": "Computer Sci

## resize:1020x1360, markdown

In [5]:
image = '/kaggle/input/psc-forms/20251111_160053.jpg'
resized_image = resize_image(image, 1024, 1360)
prompt = 'Convert to markdown'
inference(resized_image, prompt)

```markdown
REPUBLIC OF MAURITIUS

P.S.C. Form 7

FOR OFFICIAL USE

# PUBLIC SERVICE COMMISSION
## Application Form

**Read carefully the enclosed Notes & Instructions to candidates before filling this form**

---

### 1. Post Applied for: **Administrative Officer**

- Ministry/Department: **Ministry of Finance**
- Date of Advertisement/Vacancy Circular: **VC/15/2025**
- Previous Posts Applied for:
  1. **Management Analyst**
  2. **Executive Assistant**

---

### 2. National Identity No. **Q8512345678E**

- Title: **Miss** (✓)
- Marital Status: **Single** (✓)
- Surname: **Scarlett** (in block letters)
- Other Names: **Silverstone** (in block letters)
- Maiden Name (if applicable): **Not Applicable**

---

### 3. Residential Address: **NO.15, FLOWER LANE, COLOMBO 07** (in block letters)

- Phone No: Office **01234567**, Home **07712345**, Mobile **07712346**
- Email address: **Scarlett@gmail.com**
- Date of Birth: **15/08/1995**
- Age: **30**
- Place of Birth: **Colombo**
- Nationality

### resize: 895x1247, markdown

In [6]:
image = '/kaggle/input/psc-forms/20251112_100725.jpg'
resized_image = resize_image(image, 895, 1247)
prompt = 'Convert to markdown'
inference(resized_image, prompt)

```markdown
# REPUBLIC OF MAURITIUS

## PUBLIC SERVICE COMMISSION
### Application Form

**P.S.C. Form 7**
**FOR OFFICIAL USE**

---

### 1. Post Applied for:  
**Administrative officer**

- Ministry/Department: **Ministry of Education, Science and Technology**
- Date of Advertisement/Vacancy Circular: **15 September 2025**
- Previous Posts Applied for:  
  1. Clerical officer  
  2. 

---

### 2. National Identity No.  
**N22001123456789**

- Title: **Mr** (✓) Mrs Miss Ms Other:
- Marital Status: **Married** (✓) Single Other:
- Surname: **RAGGONATH** (in block letters)
- Other Names: **KAVITRESH ANAND** (in block letters)
- Maiden Name (if applicable):

---

### 3. Residential Address:  
**23 ORCHID AVENUE, VACOAS, MAURITIUS**

- Phone No.: Office **601 2055** Home **676 8921** Mobile **5784 4436** Email address: **kavirish.ragoonath6@gmail.com**
- Date of Birth: **14 03 1986** Age: **29** Place of Birth: **Rose Belle**
- Nationality: **Mauritian** Certificate No. (if Naturalised) & Da

### resize 895X1247, marksown, prompt with instructions

In [6]:
image = '/kaggle/input/psc-forms/20251112_100725.jpg'
resized_image = resize_image(image, 895, 1247)
prompt = 'Convert to markdown. make sure that the size of these fields are maintained: National identity number: 14 characters, Month/Year: 4 characters, Exam. Centre No.: 5 characters, Index No.: 5 characters'
inference(resized_image, prompt)

```markdown
# P.S.C. Form 7

## FOR OFFICIAL USE

| Height | Chest | Weight | Grading |
|--------|-------|--------|---------|
| 1.76m   | 90 cm | 72 kg  | Good    |

---

## 1. Post Applied for

**Administrative officer**

- Ministry/Department: Ministry of Education, Science and Technology
- Date of Advertisement/Vacancy Circular: 15 September 2025
- Previous Posts Applied for:
  1. Clerical officer
  2. 

---

## 2. National Identity No.

`N22001123456789` (14 characters)

- Title: Mr ☑ Mrs ☐ Miss ☐ Ms ☐ Other:
- Marital Status: Married ☑ Single ☐ Other:
- Surname: **RAGOONATH** (in block letters)
- Other Names: **KAVITRESH ANAND** (in block letters)
- Maiden Name (if applicable):

---

## 3. Residential Address

**23 ORCHID AVENUE, VACOAS, MAURITIUS** (in block letters)

- Phone No.: Office: 601 2055 | Home: 676 8921 | Mobile: 5784 44 36
- Email address: **kavirish.ragoonatha@gmail.com**
- Date of Birth: 14 03 1986 | Age: 29 | Place of Birth: Rose Belle
- Nationality: Mauritian | Ce

### resize to 1342x1870, markdown

In [5]:
image = '/kaggle/input/psc-forms/20251112_100725.jpg'
resized_image = resize_image(image, 1342, 1870)
prompt = 'Convert to markdown'
inference(resized_image, prompt)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.66 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.20 GiB is free. Process 3220 has 13.54 GiB memory in use. Of the allocated memory 13.38 GiB is allocated by PyTorch, and 30.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
image = '/kaggle/input/psc-forms/20251112_100725.jpg'
resized_image = resize_image(image, 895, 1247)
prompt = 'Retrieve applicant details. Format response in json'
inference(resized_image, prompt)

{
  "post_applied_for": "Administrative officer",
  "ministry_department": "Ministry of Education, Technology, Science and Technology",
  "date_of_advertisement": "15 September 2025",
  "previous_posts_applied_for": "Clerical officer",
  "national_identity_no": "N22001123456789",
  "title": "Mr",
  "marital_status": "Married",
  "surname": "RAGOONATH",
  "other_names": "KAVITRESH ANAND",
  "residential_address": "23 ORCHID AVENUE, VACOAS, MAURITIUS",
  "phone_no_office": "601 2055",
  "phone_no_home": "676 8921",
  "phone_no_mobile": "5784 4436",
  "email_address": "kavitresh.ragoonath6@gmail.com",
  "date_of_birth": "14 03 1986",
  "age": 29,
  "place_of_birth": "Rose Belle",
  "nationality": "Mauritian",
  "secondary_ordinary_level": {
    "certificate_type": "Cambridge school (certificate G.C.E.)",
    "exam_year": "2015",
    "exam_centre_no": "MUI 23",
    "index_no": "451 23",
    "subjects": [
      {
        "subject": "English language",
        "grade": "B"
      },
      {
 